# Hello TensorFlow

Install TensorFlow using pip:

pip install TensorFlow

Is that it?

Yes, that's all. Now you can test if you have do it ok, importing the library using:

In [1]:
import tensorflow as tf

We imported TensorFlow with the alias tf, now we can access to all the methods and classes. If you have problems importing tensorflow, consider to resintall tensorflow, use a virtual enviroment, use conda, if you are in unix like linux o MacOs, use sudo before pip, example: sudo pip install TensorFlow.

## Tensors
Tensors are multidimensional arrays, a tensor has the properties:
    - Shape
    - Datatype: int32, float32, string, etc.
Let's create variables o tensors.


In [2]:
string1 = tf.Variable('Hello1', tf.string)
string2 = tf.Variable('hello2')
int1 = tf.Variable(5)
int2 = tf.Variable(6, tf.int16)
int3 = tf.Variable(3, tf.int32)
float1 = tf.Variable(5.0)
float2 = tf.Variable(6.0, tf.float32)

Now let's print this variables

In [3]:
from __future__ import print_function

In [4]:
string1, string2

(<tf.Variable 'Variable:0' shape=() dtype=string_ref>,
 <tf.Variable 'Variable_1:0' shape=() dtype=string_ref>)

We can see the we can specify or not the type o variable, tensorflow makes the variable as a string when the string is given as parameter

In [5]:
int1, int2, int3

(<tf.Variable 'Variable_2:0' shape=() dtype=int32_ref>,
 <tf.Variable 'Variable_3:0' shape=() dtype=int32_ref>,
 <tf.Variable 'Variable_4:0' shape=() dtype=int32_ref>)

if you don't specify the type of a integer, tensorflow creates a int32 variable, integers can not be int16 in systems with 64 bits.

In [6]:
float1, float2

(<tf.Variable 'Variable_5:0' shape=() dtype=float32_ref>,
 <tf.Variable 'Variable_6:0' shape=() dtype=float32_ref>)

The same happens with the float variables

## Tensors n-dimensions

In [7]:
rank0 = tf.Variable(1)
rank1= tf.Variable([1, 2])
rank2 = tf.Variable([[1, 2],
                     [3, 4]])

We can continue creating variables with n dimensions and we can not finish
The most commons variables are rank 2, 3, and 4
- 2 when you have a data set with n features and m instances n x m, example: the iris data set
- 3 normally used when the instances are images in gray scale 
- 4 in data sets when the images are in rgb

In [8]:
rank3 = tf.zeros([10, 28, 28]) # 10 images in gray scale size 28 x 28
rank4 = tf.zeros([10, 32, 32, 3]) # 10  images in rgb size 32 x 32

## I can not see the values of my tensors

All variables are created but the values are not computed yet, to compute a value you need to create a session and initialize all variables with the next code.

In [9]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

print(sess.run([rank0, rank1, rank2]))
sess.close()

[1, array([1, 2], dtype=int32), array([[1, 2],
       [3, 4]], dtype=int32)]


## PlaceHolder
A placeholder is a variable that will get a value later, like the inputs values in a model

In [10]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
add = tf.add(x, y)

# now in a session we put this values as input

sess = tf.Session()

print(sess.run(add, {x: 5, y: 8}))

13.0


add is the value we want and x and y are placeholders and we can choose their values later.

## Create a linear model
The formule is y = m * x + b
where:
    y: is the variable we want to predict
    x: is the explanatory variable
    b: is the slope
    m: is the intercept (the value of y when x = 0)

In [11]:
w = tf.Variable([.3], tf.float32)
b = tf.Variable([-.3], tf.float32)
x = tf.placeholder(tf.float32)
yp = w * x + b

Now lets evaluate our model

In [12]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

print(sess.run(yp, {x: [1, 2, 3, 4]}))

[ 0.          0.30000001  0.60000002  0.90000004]


Now we have our model, we need to evaluate how good it is. We calculate the evaluation using the sum of the square of the deltas between yp and y provided for us.

In [13]:
y = tf.placeholder(tf.float32)
squared_deltas = tf.square(yp - y)
loss = tf.reduce_sum(squared_deltas)
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

23.66


The loss is not good, must be around 0.
## Time to train
Tensorflow privides the class train that contains optimizar to reduce the loss in our model, here we will use the Gradient Descent.

In [14]:
optimizer = tf.train.GradientDescentOptimizer(0.01)
train = optimizer.minimize(loss)
sess.run(init) # reset values to incorrect defaults.
for i in range(1000):
  sess.run(train, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print(sess.run([w, b]))

[array([-0.9999969], dtype=float32), array([ 0.99999082], dtype=float32)]


This model trained provided the correct values for w and b, these are our weights to use in the model and calculate again the loss:
the value for w is almost -1 and for b almost 1, so let's assing this values

In [15]:
fixW = tf.assign(w, [-1.])
fixb = tf.assign(b, [1.])
sess.run([fixW, fixb])
print(sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]}))

0.0


Now our loss function is 0, that means that our train almost provided the correct values for w and b

## estimator class
This class provides a linear model already made, we just feed the model with the values of x and y.

In [16]:
import numpy as np

# Our only feature is x

feature_columns = [tf.feature_column.numeric_column("x", shape=[1])]

# Create the estimator

estimator = tf.estimator.LinearRegressor(feature_columns=feature_columns)

# let's use numpy to create our train and eval values
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])

# These values will not learn the model, this is good to avoid overfitting
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

# this input is to train our model

input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, 
    y_train, 
    batch_size=4, 
    num_epochs=None, 
    shuffle=True)

# this input is to evaluate with the same values

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_train}, 
    y_train, 
    batch_size=4,
    num_epochs=1000, 
    shuffle=False)

# This input is to evaluate with the values did not learn

eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    {"x": x_eval}, 
    y_eval, batch_size=4, 
    num_epochs=1000, 
    shuffle=False)

# Here the model train

estimator.train(input_fn=input_fn, steps=1000)

# Here we evaluate how well our model did.

# With the train inputs
train_metrics = estimator.evaluate(input_fn=train_input_fn)

# With the eval inputs
eval_metrics = estimator.evaluate(input_fn=eval_input_fn)

print("train metrics: %r"% train_metrics)
print("eval metrics: %r"% eval_metrics)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_tf_random_seed': 1, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_save_checkpoints_steps': None, '_model_dir': '/tmp/tmpjzDWsg', '_save_summary_steps': 100}
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into /tmp/tmpjzDWsg/model.ckpt.
INFO:tensorflow:loss = 2.0, step = 1
INFO:tensorflow:global_step/sec: 829.127
INFO:tensorflow:loss = 0.0857892, step = 101 (0.125 sec)
INFO:tensorflow:global_step/sec: 898.852
INFO:tensorflow:loss = 0.0275062, step = 201 (0.108 sec)
INFO:tensorflow:global_step/sec: 1326.96
INFO:tensorflow:loss = 0.00755187, step = 301 (0.076 sec)
INFO:tensorflow:global_step/sec: 842.226
INFO:tensorflow:loss = 0.000955762, step = 401 (0.119 sec)
INFO:tensorflow:global_step/sec: 960.762
INFO:tensorflow:loss = 0.000124974, step = 501 (0.104 sec)
INFO:tensorfl

We can see that with the train metrics the loss is almost 0 and less than the eval metrics loss, but the important results are from the eval metrics, because evaluate inputs that the model did not learn